In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import json
import tqdm
import datetime

df_2 = pd.read_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\ACLED_GOSIF_GPP_CHIRPS_GOSIF_GPP_SD_FAO.csv')
# convert geo_df date to datetime
df_2['date'] = pd.to_datetime(df_2['date'])
from shapely import wkt

# Convert the 'geometry' column to shapely geometry
df_2['geometry'] = df_2['geometry'].apply(wkt.loads)

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df_2, geometry='geometry')

# find the centroid of each polygon
gdf['centroid'] = gdf['geometry'].centroid

# extract the lat and lon from the centroid
gdf['lon'] = gdf['centroid'].apply(lambda p: p.x)
gdf['lat'] = gdf['centroid'].apply(lambda p: p.y)


new_df = gdf
# extract distinct lat and lon, and create a new df
lat_lon = new_df[['lat', 'lon']].drop_duplicates().reset_index(drop=True)
# round to two digits, and create a new column
lat_lon['lat_round'] = lat_lon['lat'].round(2)
lat_lon['lon_round'] = lat_lon['lon'].round(2)


In [10]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session



# create a new df to store the data
result = pd.DataFrame(columns=['date', 'lat', 'lon', 'temperature_2m_mean', 'temperature_2m_mean_sd'])

# loop through each lat and lon
for i in tqdm.tqdm(range(len(lat_lon))):
    # request the data from the API
    url = 'https://archive-api.open-meteo.com/v1/archive?latitude=' + str(lat_lon['lat_round'][i]) + '&longitude=' + str(lat_lon['lon_round'][i]) + '&start_date=2017-01-01&end_date=2022-12-31&models=era5&daily=temperature_2m_mean&timezone=America%2FNew_York'
    try:
        response = requests_retry_session().get(url)
        response.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)
    data = response.json()
    # create a new df to store the data
    df = pd.DataFrame(columns=['date', 'lat', 'lon', 'temperature_2m_mean'])
    # extract time and temperature_2m_mean from the json file
    df['date'] = data['daily']['time']
    df['temperature_2m_mean'] = data['daily']['temperature_2m_mean']
    df['lat'] = lat_lon['lat'][i]
    df['lon'] = lat_lon['lon'][i]
    # convert date to datetime
    df['date'] = pd.to_datetime(df['date'])
    # calculate montely average and sd
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['month_year'] = df['month'].astype(str) + '-' + df['year'].astype(str)
    df['month_year'] = pd.to_datetime(df['month_year'])
    df['month_year'] = df['month_year'].dt.strftime('%Y-%m')
    df = df.groupby(['month_year', 'lat', 'lon']).agg({'temperature_2m_mean': ['mean', 'std']}).reset_index()
    df.columns = ['month_year', 'lat', 'lon', 'temperature_2m_mean', 'temperature_2m_mean_sd']
    # rename month_year to date
    df = df.rename(columns={'month_year': 'date'})
    # concat the data
    result = pd.concat([result, df], axis=0)
    # sleep for 1 second
    time.sleep(1)



  0%|          | 0/1880 [00:00<?, ?it/s]C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_22596\3715194460.py:58: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['month_year'] = pd.to_datetime(df['month_year'])
  0%|          | 1/1880 [00:01<48:22,  1.54s/it]C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_22596\3715194460.py:58: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['month_year'] = pd.to_datetime(df['month_year'])
  0%|          | 2/1880 [00:03<48:34,  1.55s/it]C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_22596\3715194460.py:58: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specif

In [12]:
result.describe()

,lat,lon,temperature_2m_mean,temperature_2m_mean_sd
count,135360.000000,135360.000000,135360.000000,135360.000000
mean,3.817693,29.385065,23.691101,1.276133
std,14.650756,31.655686,5.597562,0.717369
min,-34.041425,-91.931535,-22.016129,0.177393
25%,-4.705279,26.566166,21.216129,0.814383
50%,4.978378,33.984001,24.774194,1.080916
75%,13.978251,44.204766,26.987644,1.513654
max,38.108696,71.597844,39.946667,8.627638


In [14]:
gdf.dtypes

date                      datetime64[ns]
geometry                        geometry
event_count_battles              float64
event_count_explosions           float64
event_count_violence             float64
fatalities_battles               float64
fatalities_explosions            float64
fatalities_violence              float64
GOSIF_GPP                        float64
rainfall_chirps                  float64
GOSIF_GPP_SD                     float64
elevation                        float64
soil                             float64
price                            float64
centroid                        geometry
lon                              float64
lat                              float64
dtype: object

In [16]:
# convert date in result to datetime
result['date'] = pd.to_datetime(result['date'])

In [17]:




#merge the data
gdf_merged = pd.merge(gdf, result, on=['date', 'lat', 'lon'], how='left')


In [20]:
# read csv
short_wav = pd.read_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\shortwave_radiation_sum.csv')


In [21]:
short_wav['date'] = pd.to_datetime(short_wav['date'])

In [23]:
gdf_merged.dtypes

date                      datetime64[ns]
geometry                        geometry
event_count_battles              float64
event_count_explosions           float64
event_count_violence             float64
fatalities_battles               float64
fatalities_explosions            float64
fatalities_violence              float64
GOSIF_GPP                        float64
rainfall_chirps                  float64
GOSIF_GPP_SD                     float64
elevation                        float64
soil                             float64
price                            float64
centroid                        geometry
lon                              float64
lat                              float64
temperature_2m_mean              float64
temperature_2m_mean_sd           float64
dtype: object

In [24]:

#merge the data
gdf_merged_new = pd.merge(gdf_merged, short_wav, on=['date', 'lat', 'lon'], how='left')

In [25]:
# drop centroid
gdf_merged_new = gdf_merged_new.drop(['centroid'], axis=1)

In [26]:
# describe
gdf_merged_new.describe()

,date,event_count_battles,event_count_explosions,event_count_violence,fatalities_battles,fatalities_explosions,fatalities_violence,GOSIF_GPP,rainfall_chirps,GOSIF_GPP_SD,elevation,soil,price,lon,lat,temperature_2m_mean,temperature_2m_mean_sd,shortwave_radiation_sum,shortwave_radiation_sum_sd
count,40881,40881.000000,40881.000000,40881.000000,40881.000000,40881.000000,40881.000000,40318.000000,38063.000000,37978.000000,6799.000000,3243.000000,6379.000000,40881.000000,40881.000000,40881.000000,40881.000000,29948.000000,29948.000000
mean,2020-01-06 00:03:25.004770048,3.285658,1.603777,1.573959,10.312615,3.501676,2.430420,10117.492916,64.829655,1308.441980,641.392411,137.747456,109.354167,30.864571,9.894532,23.566191,1.304132,20.615955,2.734613
min,2017-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,98.071429,0.000041,612.000000,0.000000,105.000000,7.755585,-91.931535,-34.041425,-21.500000,0.179605,7.384516,0.203383
25%,2018-07-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2864.779937,6.223252,961.507576,284.000000,133.000000,81.834942,28.846987,2.737897,20.833333,0.757358,18.660323,1.836812
50%,2020-02-01 00:00:00,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,6430.990356,28.256059,1145.380952,478.000000,136.000000,97.470534,42.835959,7.802179,25.183333,1.041021,20.722667,2.652295
75%,2021-07-01 00:00:00,3.000000,1.000000,2.000000,6.000000,0.000000,2.000000,17505.955335,102.775804,1389.247059,950.000000,147.000000,120.107663,46.124955,15.017249,27.754839,1.559163,22.802981,3.517416
max,2022-12-01 00:00:00,141.000000,99.000000,117.000000,897.000000,624.000000,380.000000,33774.707702,1221.385403,6544.823529,2492.000000,150.000000,683.228457,71.597844,38.108696,38.632258,8.627638,31.027667,10.062772
std,NaN,7.821836,4.883601,3.459235,34.104102,15.671332,11.171727,8748.553396,80.508854,566.464791,539.251303,8.867594,57.962848,38.870069,13.409458,6.716554,0.851394,3.650981,1.200064


In [27]:
# save to csv
gdf_merged_new.to_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\comprehensive.csv', index=False)